In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import pickle



pd.set_option('display.max_columns', None)

In [108]:
data = pd.read_csv("/content/updated - finished dataset - in (5) (1).csv")
data.isnull()

,ID,Age,Gender,maxillaryR score,maxillary R gray/black ratio,m_r_std,m_r_1,m_r_2,m_r_3,m_r_4,m_r_5,maxillary L score,maxillary L gray/black ratio,m_l_std,m_l_1,m_l_2,m_l_3,m_l_4,m_l_5,frontal score,frontal gray/black ratio,front_std,front_1,front_2,front_3,front_4,front_5,ant.ethmoid R score,ant.ethmoid R gray/black ratio,ant_r_std,ant_r_1,ant_r_2,ant_r_3,ant_r_4,ant_r_5,ant.ethmoid L score,ant.ethmoid L gray/black ratio,ant_l_std,ant_l_1,ant_l_2,ant_l_3,ant_l_4,ant_l_5,post.ethmoid R score,post.ethmoid R gray/black ratio,post_r_std,post_r_1,post_r_2,post_r_3,post_r_4,post_r_5,post.ethmoid L score,post.ethmoid L gray/black ratio,post_l_std,post_l_1,post_l_2,post_l_3,post_l_4,post_l_5,sphenoid R score,sphenoid R gray/black ratio,sphen_r_std,sphen_r_1,sphen_r_2,sphen_r_3,sphen_r_4,sphen_r_5,sphenoid L score,sphenoid L gray/black ratio,sphen_l_std,sphen_l_1,sphen_l_2,sphen_l_3,sphen_l_4,sphen_l_5,Osteomeatal complex R score,Osteomeatal complex L score,septum score,septum area,sept_std,sept_1,sept_2,sept_3,sept_4,sept_5,inferior turbinate score,inferior turbinate ratio,it_std,it_1,it_2,it_3,it_4,it_5,concha bullosa score,concha_blk_ratio,concha_blk_std,concha_blk_1,concha_blk_2,concha_blk_3,concha_blk_4,concha_blk_5,concha_size_ratio,concha_size_std,concha_size_1,concha_size_2,concha_size_3,concha_size_4,concha_size_5,nasopharynx score,nasopharynx area,naso_std,naso_1,naso_2,naso_3,naso_4,naso_5
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [109]:
data.drop('ID', inplace=True, axis=1)
data.drop('Age', inplace=True, axis=1)
data.drop('Gender', inplace=True, axis=1)

In [110]:
data.loc[data['maxillaryR score'] == 1, 'maxillaryR score'] = 2
data.loc[data['maxillary L score'] == 1, 'maxillary L score'] = 2
data.loc[data['frontal score'] == 1, 'frontal score'] = 2
data.loc[data['sphenoid R score'] == 1, 'sphenoid R score'] = 2
data.loc[data['sphenoid L score'] == 1, 'sphenoid L score'] = 2

In [111]:
sinuses_score=data['maxillaryR score']+data['maxillary L score']+data['ant.ethmoid R score']+data['ant.ethmoid L score']+data['post.ethmoid R score']+data['post.ethmoid L score']+data['frontal score']+data['frontal score']+data['sphenoid R score']+data['sphenoid L score']+data['Osteomeatal complex R score']+data['Osteomeatal complex L score']
data['sinuses_score'] = sinuses_score
data['sinuses_score'] = data['sinuses_score'].apply(lambda x: 0 if x < 18 else 2)



In [112]:
data['sergury'] = 0
data.loc[data['sinuses_score'] ==2, 'sergury'] = 2
data.loc[data['septum score'] ==2, 'sergury'] = 2
data.loc[data['inferior turbinate score'] ==2, 'sergury'] = 2
data.loc[data['concha bullosa score'] ==2, 'sergury'] = 2
data.loc[data['nasopharynx score'] ==2, 'sergury'] = 2

#data spliting into train and test

In [113]:
X = data.drop('sergury', axis=1)
y = data['sergury']
X_train, X_test, y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)
data = pd.concat([pd.DataFrame(X_train, columns=X.columns), pd.Series(y_train, name="sergury")], axis=1)

In [ ]:
X_train.shape

(211, 114)

#training_models subsets

##df_septum

In [ ]:
septum_df_col=['septum_score','septum_area','septum_std','septum_1','septum_2','septum_3','septum_4','septum_5',
               'inferior_score','inferior_ratio','inferior_std','concha_score','concha_size_ratio','concha_size_std']

septum_data_col=['septum score','septum area','sept_std','sept_1','sept_2','sept_3','sept_4','sept_5',
               'inferior turbinate score','inferior turbinate ratio','it_std','concha bullosa score','concha_size_ratio','concha_size_std']

df_septum = pd.DataFrame()

for (i, n) in zip(septum_df_col, septum_data_col):
  df_septum[i]= X_train[n]


df_septum.reset_index(inplace=True)

##df_inferior

In [ ]:
inferior_df_col=['inferior_score','inferior_ratio','inferior_std','inferior_1','inferior_2','inferior_3','inferior_4','inferior_5']

inferior_data_col=['inferior turbinate score','inferior turbinate ratio','it_std','it_1','it_2','it_3','it_4','it_5']

df_inferior = pd.DataFrame()

for (i, n) in zip(inferior_df_col, inferior_data_col):
  df_inferior[i]= data[n]

df_inferior.reset_index(inplace=True)

##df_nasopharynx

In [ ]:
nasopharynx_df_col=['nasopharynx_score','nasopharynx_area','naso_std','naso_1','naso_2','naso_3','naso_4','naso_5']

nasopharynx_data_col=['nasopharynx score','nasopharynx area','naso_std','naso_1','naso_2','naso_3','naso_4','naso_5']

df_nasopharynx = pd.DataFrame()

for (i, n) in zip(nasopharynx_df_col, nasopharynx_data_col):
  df_nasopharynx[i]= data[n]

df_nasopharynx.reset_index(inplace=True)


##df_concha

In [ ]:
concha_df_col=['concha_score','concha_blk_ratio','concha_blk_std','concha_blk_1','concha_blk_2','concha_blk_3','concha_blk_4','concha_blk_5',
                    'concha_size_ratio','concha_size_std','concha_size_1','concha_size_2','concha_size_3','concha_size_4','concha_size_5']

concha_data_col=['concha bullosa score','concha_blk_ratio','concha_blk_std','concha_blk_1','concha_blk_2','concha_blk_3','concha_blk_4','concha_blk_5',
                 'concha_size_ratio','concha_size_std','concha_size_1','concha_size_2','concha_size_3','concha_size_4','concha_size_5']

df_concha = pd.DataFrame()

for (i, n) in zip(concha_df_col, concha_data_col):
  df_concha[i]= data[n]

df_concha.reset_index(inplace=True)


##df_Osteomeatal

In [ ]:
Osteomeatal_df_col=['osteomeatal_score','maxillary_ratio','maxillary_std','ant_ethmoid_ratio','ant_ethmoid_std','maxillary_score','ant_ethmoid_score']

Osteomeatal_r_data_col=['Osteomeatal complex R score','maxillary R gray/black ratio','m_r_std','ant.ethmoid R gray/black ratio',
                        'ant_r_std','maxillaryR score','ant.ethmoid R score']

Osteomeatal_l_data_col=['Osteomeatal complex L score','maxillary L gray/black ratio','m_l_std','ant.ethmoid L gray/black ratio',
                        'ant_l_std','maxillary L score','ant.ethmoid L score']

df_Osteomeatal = pd.DataFrame()

for (i, n, z) in zip(Osteomeatal_df_col, Osteomeatal_r_data_col,Osteomeatal_l_data_col):
  df_Osteomeatal[i]= pd.concat([data[n], data[z]])



df_Osteomeatal.reset_index(inplace=True)

##df_frontal

In [ ]:
frontal_df_col=['frontal_score','frontal_ratio','frontal_std','frontal_1','frontal_2','frontal_3','frontal_4','frontal_5']

frontal_data_col=['frontal score','frontal gray/black ratio','front_std','front_1','front_2','front_3','front_4','front_5']


df_frontal = pd.DataFrame()

for (i, n) in zip(frontal_df_col, frontal_data_col):
  df_frontal[i]= data[n]

df_frontal.reset_index(inplace=True)
# df_frontal

##df_sphenoid

In [ ]:
sphenoid_df_col=['sphenoid_score','sphenoid_ratio','sphenoid_std','sphenoid_1','sphenoid_2','sphenoid_3','sphenoid_4','sphenoid_5']

sphenoid_r_data_col=['sphenoid R score','sphenoid R gray/black ratio','sphen_r_std','sphen_r_1','sphen_r_2','sphen_r_3','sphen_r_4','sphen_r_5']

sphenoid_l_data_col=['sphenoid L score','sphenoid L gray/black ratio','sphen_l_std','sphen_l_1','sphen_l_2','sphen_l_3','sphen_l_4','sphen_l_5']



df_sphenoid = pd.DataFrame()

for (i, n, z) in zip(sphenoid_df_col, sphenoid_r_data_col, sphenoid_l_data_col):
  df_sphenoid[i]= pd.concat([data[n], data[z]])

df_sphenoid.reset_index(inplace=True)


##df_post_ethmoid

In [ ]:
post_df_col=['post_ethmoid_score','post_ethmoid_ratio','post_ethmoid_std','post_ethmoid_1','post_ethmoid_2','post_ethmoid_3','post_ethmoid_4','post_ethmoid_5']

post_r_data_col=['post.ethmoid R score','post.ethmoid R gray/black ratio','post_r_std','post_r_1','post_r_2','post_r_3','post_r_4','post_r_5']

post_l_data_col=['post.ethmoid L score','post.ethmoid L gray/black ratio','post_l_std','post_l_1','post_l_2','post_l_3','post_l_4','post_l_5']


df_post_ethmoid = pd.DataFrame()

for (i, n, z) in zip(post_df_col, post_r_data_col, post_l_data_col):
  df_post_ethmoid[i]= pd.concat([data[n], data[z]])


df_post_ethmoid.reset_index(inplace=True)

##df_ant_ethmoid

In [ ]:
ant_df_col=['ant_ethmoid_score','ant_ethmoid_ratio','ant_ethmoid_std','ant_ethmoid_1','ant_ethmoid_2','ant_ethmoid_3','ant_ethmoid_4','ant_ethmoid_5']

ant_r_data_col=['ant.ethmoid R score','ant.ethmoid R gray/black ratio','ant_r_std','ant_r_1','ant_r_2','ant_r_3','ant_r_4','ant_r_5']

ant_l_data_col=['ant.ethmoid L score','ant.ethmoid L gray/black ratio','ant_l_std','ant_l_1','ant_l_2','ant_l_3','ant_l_4','ant_l_5']


df_ant_ethmoid = pd.DataFrame()

for (i, n, z) in zip(ant_df_col, ant_r_data_col, ant_l_data_col):
  df_ant_ethmoid[i]= pd.concat([data[n], data[z]])

df_ant_ethmoid.reset_index(inplace=True)

##df_maxillary

In [ ]:
maxillary_df_col=['maxillary_score','maxillary_ratio','maxillary_std','maxillary_1','maxillary_2','maxillary_3','maxillary_4','maxillary_5']

maxillary_r_data_col=['maxillaryR score','maxillary R gray/black ratio','m_r_std','m_r_1','m_r_2','m_r_3','m_r_4','m_r_5']

maxillary_l_data_col=['maxillary L score','maxillary L gray/black ratio','m_l_std','m_l_1','m_l_2','m_l_3','m_l_4','m_l_5']


df_maxillary = pd.DataFrame()

for (i, n, z) in zip(maxillary_df_col, maxillary_r_data_col, maxillary_l_data_col):
  df_maxillary[i]= pd.concat([data[n], data[z]])

df_maxillary.reset_index(inplace=True)

#lists of data subsets and their labels

In [ ]:
list_of_datasets = [df_maxillary, df_ant_ethmoid, df_post_ethmoid,df_sphenoid,df_frontal,df_Osteomeatal,df_concha,df_nasopharynx,df_inferior,df_septum]
labels=['maxillary_score','ant_ethmoid_score','post_ethmoid_score','sphenoid_score','frontal_score','osteomeatal_score','concha_score','nasopharynx_score','inferior_score','septum_score']

#balancing the subsets

In [ ]:
for df, label_name in zip(list_of_datasets, labels):
    print(df[label_name].value_counts())
    print()

0    286
2    136
Name: maxillary_score, dtype: int64

0    278
2    144
Name: ant_ethmoid_score, dtype: int64

0    301
2    121
Name: post_ethmoid_score, dtype: int64

0    359
2     63
Name: sphenoid_score, dtype: int64

0    168
2     43
Name: frontal_score, dtype: int64

0    318
2    104
Name: osteomeatal_score, dtype: int64

0    119
2     92
Name: concha_score, dtype: int64

0    185
2     26
Name: nasopharynx_score, dtype: int64

2    135
0     76
Name: inferior_score, dtype: int64

0    119
2     92
Name: septum_score, dtype: int64



In [ ]:
for index, label in enumerate(labels):
  # if(label in ('frontal_score','nasopharynx_score')):
    dataset = list_of_datasets[index]
    X = dataset.drop(label, axis=1)
    y = dataset[label]

    smote = SMOTE()
    X_resampled, y_resampled = smote.fit_resample(X, y)

    updated_dataset = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name=label)], axis=1)

    list_of_datasets[index] = updated_dataset

    print(updated_dataset[label].value_counts())



0    286
2    286
Name: maxillary_score, dtype: int64
0    278
2    278
Name: ant_ethmoid_score, dtype: int64
0    301
2    301
Name: post_ethmoid_score, dtype: int64
0    359
2    359
Name: sphenoid_score, dtype: int64
0    168
2    168
Name: frontal_score, dtype: int64
0    318
2    318
Name: osteomeatal_score, dtype: int64
2    119
0    119
Name: concha_score, dtype: int64
0    185
2    185
Name: nasopharynx_score, dtype: int64
2    135
0    135
Name: inferior_score, dtype: int64
2    119
0    119
Name: septum_score, dtype: int64


#models training

In [ ]:
trained_models = {}
model_key=['maxillary','ant_ethmoid','post_ethmoid','sphenoid','frontal','osteomeatal','concha','nasopharynx','inferior','septum']

In [73]:
param_grid = {
    'n_estimators': [10,20,50,100, 200, 300, 500],
    'max_depth': [None, 5, 7,8,10],
    'min_samples_split': [2, 5, 10, 20,50],
    # 'learning_rate': [0.1, 0.01, 0.001, 0.0001]
}

for df, label_name, key in zip(list_of_datasets, labels, model_key):
    best_accuracy = 0.0
    best_model = None
    best_val_predictions = None

    print(label_name)

    X = df.drop(label_name, axis=1)
    y = df[label_name]

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    # Iterate over hyperparameter combinations
    for n_estimators in param_grid['n_estimators']:
        for max_depth in param_grid['max_depth']:
            for min_samples_split in param_grid['min_samples_split']:
                # Create an instance of the Random Forest model with the current hyperparameters
                model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split)

                # Perform cross-validation on the training data
                scores = cross_val_score(model, X_train, y_train, cv=5) 

                # Compute the mean accuracy across all cross-validation folds
                accuracy = scores.mean()

                # Check if this model has better accuracy than the previous best model
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_model = model

                    # Train the best model on the entire training data
                    best_model.fit(X_train, y_train)
                    trained_models[key] = best_model

                    # Make predictions on the validation data
                    best_val_predictions = best_model.predict(X_val)


    print("Best Accuracy:", best_accuracy)
    print()


maxillary_score
Best Accuracy: 0.8971571906354514

ant_ethmoid_score
Best Accuracy: 0.896552604698672

post_ethmoid_score
Best Accuracy: 0.9064218213058419

sphenoid_score
Best Accuracy: 0.9355606407322654

frontal_score
Best Accuracy: 0.9514325646401118

osteomeatal_score
Best Accuracy: 0.9193166375461075

concha_score
Best Accuracy: 0.6684210526315789

nasopharynx_score
Best Accuracy: 0.8986440677966103

inferior_score
Best Accuracy: 0.8470401691331924

septum_score
Best Accuracy: 0.668421052631579



#training AUC and reports

In [74]:
X_train = data.drop("sergury", axis=1)
y_train = data["sergury"]

##maxillary_prediction

In [75]:
df_maxillary = pd.DataFrame()

for (i, n) in zip(maxillary_df_col[1:], maxillary_r_data_col[1:]):
  df_maxillary[i]= X_train[n]

df_maxillary.reset_index(inplace=True)

m_r_p=trained_models['maxillary'].predict(df_maxillary)

#-----------------------------------------------------------------------------------

df_maxillary = pd.DataFrame()

for (i, n) in zip(maxillary_df_col[1:], maxillary_l_data_col[1:]):
  df_maxillary[i]= X_train[n]

df_maxillary.reset_index(inplace=True)

m_l_p=trained_models['maxillary'].predict(df_maxillary)


##frontal_prediction

In [76]:
df_frontal = pd.DataFrame()

for (i, n) in zip(frontal_df_col[1:], frontal_data_col[1:]):
  df_frontal[i]= X_train[n]

df_frontal.reset_index(inplace=True)

f_p=m_l_p=trained_models['frontal'].predict(df_frontal)


##ant_prediction

In [77]:
df_ant_ethmoid = pd.DataFrame()

for (i, n) in zip(ant_df_col[1:], ant_r_data_col[1:]):
  df_ant_ethmoid[i]= X_train[n]

df_ant_ethmoid.reset_index(inplace=True)

ant_r_p=trained_models['ant_ethmoid'].predict(df_ant_ethmoid)

#------------------------------------------------------------------------------------------

df_ant_ethmoid = pd.DataFrame()

for (i, n) in zip(ant_df_col[1:], ant_l_data_col[1:]):
  df_ant_ethmoid[i]= X_train[n]

df_ant_ethmoid.reset_index(inplace=True)  

ant_l_p=trained_models['ant_ethmoid'].predict(df_ant_ethmoid)

##post_prediction

In [78]:
df_post_ethmoid = pd.DataFrame()

for (i, n) in zip(post_df_col[1:], post_r_data_col[1:]):
  df_post_ethmoid[i]= X_train[n]

df_post_ethmoid.reset_index(inplace=True)  

post_r_p=trained_models['post_ethmoid'].predict(df_post_ethmoid)

#------------------------------------------------------------------------------------------

df_post_ethmoid = pd.DataFrame()

for (i, n) in zip(post_df_col[1:], post_l_data_col[1:]):
  df_post_ethmoid[i]= X_train[n]

df_post_ethmoid.reset_index(inplace=True)  

post_l_p=trained_models['post_ethmoid'].predict(df_post_ethmoid)

##sphenoid_prediction

In [79]:
df_sphenoid = pd.DataFrame()

for (i, n) in zip(sphenoid_df_col[1:], sphenoid_r_data_col[1:]):
  df_sphenoid[i]= X_train[n]

df_sphenoid.reset_index(inplace=True)

sphen_r_p=trained_models['sphenoid'].predict(df_sphenoid)

#-----------------------------------------------------------------------------

df_sphenoid = pd.DataFrame()

for (i, n) in zip(sphenoid_df_col[1:], sphenoid_l_data_col[1:]):
  df_sphenoid[i]= X_train[n]

df_sphenoid.reset_index(inplace=True)

sphen_l_p=trained_models['sphenoid'].predict(df_sphenoid)

##Osteomeatal_prediction

In [80]:
df_Osteomeatal = pd.DataFrame()

for (i, n) in zip(Osteomeatal_df_col[1:], Osteomeatal_r_data_col[1:]):
  df_Osteomeatal[i]= X_train[n]

df_Osteomeatal['ant_ethmoid_score']=ant_r_p
df_Osteomeatal['maxillary_score']=m_r_p

df_Osteomeatal.reset_index(inplace=True)

ost_r_p=trained_models['osteomeatal'].predict(df_Osteomeatal)

#------------------------------------------------------------------

df_Osteomeatal = pd.DataFrame()

for (i, n) in zip(Osteomeatal_df_col[1:], Osteomeatal_l_data_col[1:]):
  df_Osteomeatal[i]= X_train[n]

df_Osteomeatal['ant_ethmoid_score']=ant_l_p
df_Osteomeatal['maxillary_score']=m_l_p

df_Osteomeatal.reset_index(inplace=True)

ost_l_p=trained_models['osteomeatal'].predict(df_Osteomeatal)


##concha_prediction

In [81]:
df_concha = pd.DataFrame()

for (i, n) in zip(concha_df_col[1:], concha_data_col[1:]):
  df_concha[i]= X_train[n]

df_concha.reset_index(inplace=True)


concha_p=trained_models['concha'].predict(df_concha)

##nasopharynx_prediction

In [82]:
df_nasopharynx = pd.DataFrame()

for (i, n) in zip(nasopharynx_df_col[1:], nasopharynx_data_col[1:]):
  df_nasopharynx[i]= X_train[n]

df_nasopharynx.reset_index(inplace=True)

naso_p=trained_models['nasopharynx'].predict(df_nasopharynx)

##inferior_prediction

In [83]:
df_inferior = pd.DataFrame()

for (i, n) in zip(inferior_df_col[1:], inferior_data_col[1:]):
  df_inferior[i]= X_train[n]

df_inferior.reset_index(inplace=True)

infer_p=trained_models['inferior'].predict(df_inferior)

##septum_prediction

In [84]:
df_septum = pd.DataFrame()

for (i, n) in zip(septum_df_col[1:], septum_data_col[1:]):
  df_septum[i]= X_train[n]

df_septum['inferior_score']= infer_p
df_septum['concha_score']= concha_p


df_septum.reset_index(inplace=True)


sept_p=trained_models['septum'].predict(df_septum)

##final_prediction_dataset

In [85]:
prediction = pd.DataFrame()
sinuses_score_p=m_r_p+m_l_p+ant_r_p+ant_l_p+post_r_p+post_l_p+sphen_r_p+sphen_l_p+ost_r_p+ost_l_p+f_p
prediction['sinuses_score_p'] = sinuses_score_p
prediction['sinuses_score_p'] = prediction['sinuses_score_p'].apply(lambda x: 0 if x < 18 else 2)
prediction['sergury_p'] = 0
prediction['sept_p']=sept_p
prediction['infer_p']=infer_p
prediction['concha_p']=concha_p
prediction['naso_p']=naso_p
prediction.loc[prediction['sinuses_score_p'] ==2, 'sergury_p'] = 2
prediction.loc[prediction['naso_p'] ==2, 'sergury_p'] = 2
prediction.loc[prediction['infer_p'] ==2, 'sergury_p'] = 2
prediction.loc[prediction['concha_p'] ==2, 'sergury_p'] = 2
prediction.loc[prediction['naso_p'] ==2, 'sergury_p'] = 2

In [86]:
prediction.head(3)

,sinuses_score_p,sergury_p,sept_p,infer_p,concha_p,naso_p
0,0,2,2,0,2,0
1,0,2,0,2,2,0
2,0,2,2,2,2,0


##training reports

In [87]:
report = classification_report(X_train['sinuses_score'], prediction['sinuses_score_p'])
print(report)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       177
           2       0.94      0.94      0.94        34

    accuracy                           0.98       211
   macro avg       0.96      0.96      0.96       211
weighted avg       0.98      0.98      0.98       211



In [88]:
report = classification_report(X_train['septum score'], prediction['sept_p'])
print(report)

              precision    recall  f1-score   support

           0       0.94      0.92      0.93       119
           2       0.90      0.92      0.91        92

    accuracy                           0.92       211
   macro avg       0.92      0.92      0.92       211
weighted avg       0.92      0.92      0.92       211



In [89]:
report = classification_report(X_train['inferior turbinate score'], prediction['infer_p'])
print(report)

              precision    recall  f1-score   support

           0       0.81      0.74      0.77        76
           2       0.86      0.90      0.88       135

    accuracy                           0.84       211
   macro avg       0.84      0.82      0.83       211
weighted avg       0.84      0.84      0.84       211



In [90]:
report = classification_report(X_train['concha bullosa score'], prediction['concha_p'])
print(report)

              precision    recall  f1-score   support

           0       0.72      0.98      0.83       119
           2       0.96      0.51      0.67        92

    accuracy                           0.78       211
   macro avg       0.84      0.75      0.75       211
weighted avg       0.83      0.78      0.76       211



In [91]:
report = classification_report(X_train['nasopharynx score'], prediction['naso_p'])
print(report)

              precision    recall  f1-score   support

           0       1.00      0.96      0.98       185
           2       0.76      1.00      0.87        26

    accuracy                           0.96       211
   macro avg       0.88      0.98      0.92       211
weighted avg       0.97      0.96      0.96       211



##training AUC

In [92]:
y_p=prediction['sergury_p']
training_auc = roc_auc_score(y_train, y_p)

print("training auc:", training_auc)

training auc: 0.7961348995831755


#testing AUC and reports

##maxillary_prediction

In [114]:
df_maxillary = pd.DataFrame()

for (i, n) in zip(maxillary_df_col[1:], maxillary_r_data_col[1:]):
  df_maxillary[i]= X_test[n]

df_maxillary.reset_index(inplace=True)

m_r_p=trained_models['maxillary'].predict(df_maxillary)

#-----------------------------------------------------------------------------------

df_maxillary = pd.DataFrame()

for (i, n) in zip(maxillary_df_col[1:], maxillary_l_data_col[1:]):
  df_maxillary[i]= X_test[n]

df_maxillary.reset_index(inplace=True)

m_l_p=trained_models['maxillary'].predict(df_maxillary)


##frontal_prediction

In [115]:
df_frontal = pd.DataFrame()

for (i, n) in zip(frontal_df_col[1:], frontal_data_col[1:]):
  df_frontal[i]= X_test[n]

df_frontal.reset_index(inplace=True)

f_p=m_l_p=trained_models['frontal'].predict(df_frontal)


##ant_prediction

In [116]:
df_ant_ethmoid = pd.DataFrame()

for (i, n) in zip(ant_df_col[1:], ant_r_data_col[1:]):
  df_ant_ethmoid[i]= X_test[n]

df_ant_ethmoid.reset_index(inplace=True)

ant_r_p=trained_models['ant_ethmoid'].predict(df_ant_ethmoid)

#------------------------------------------------------------------------------------------

df_ant_ethmoid = pd.DataFrame()

for (i, n) in zip(ant_df_col[1:], ant_l_data_col[1:]):
  df_ant_ethmoid[i]= X_test[n]

df_ant_ethmoid.reset_index(inplace=True)  

ant_l_p=trained_models['ant_ethmoid'].predict(df_ant_ethmoid)

##post_prediction

In [117]:
df_post_ethmoid = pd.DataFrame()

for (i, n) in zip(post_df_col[1:], post_r_data_col[1:]):
  df_post_ethmoid[i]= X_test[n]

df_post_ethmoid.reset_index(inplace=True)  

post_r_p=trained_models['post_ethmoid'].predict(df_post_ethmoid)

#------------------------------------------------------------------------------------------

df_post_ethmoid = pd.DataFrame()

for (i, n) in zip(post_df_col[1:], post_l_data_col[1:]):
  df_post_ethmoid[i]= X_test[n]

df_post_ethmoid.reset_index(inplace=True)  

post_l_p=trained_models['post_ethmoid'].predict(df_post_ethmoid)

##sphenoid_prediction

In [118]:
df_sphenoid = pd.DataFrame()

for (i, n) in zip(sphenoid_df_col[1:], sphenoid_r_data_col[1:]):
  df_sphenoid[i]= X_test[n]

df_sphenoid.reset_index(inplace=True)

sphen_r_p=trained_models['sphenoid'].predict(df_sphenoid)

#-----------------------------------------------------------------------------

df_sphenoid = pd.DataFrame()

for (i, n) in zip(sphenoid_df_col[1:], sphenoid_l_data_col[1:]):
  df_sphenoid[i]= X_test[n]

df_sphenoid.reset_index(inplace=True)

sphen_l_p=trained_models['sphenoid'].predict(df_sphenoid)

##Osteomeatal_prediction

In [119]:
df_Osteomeatal = pd.DataFrame()

for (i, n) in zip(Osteomeatal_df_col[1:], Osteomeatal_r_data_col[1:]):
  df_Osteomeatal[i]= X_test[n]

df_Osteomeatal['ant_ethmoid_score']=ant_r_p
df_Osteomeatal['maxillary_score']=m_r_p

df_Osteomeatal.reset_index(inplace=True)

ost_r_p=trained_models['osteomeatal'].predict(df_Osteomeatal)

#------------------------------------------------------------------

df_Osteomeatal = pd.DataFrame()

for (i, n) in zip(Osteomeatal_df_col[1:], Osteomeatal_l_data_col[1:]):
  df_Osteomeatal[i]= X_test[n]

df_Osteomeatal['ant_ethmoid_score']=ant_l_p
df_Osteomeatal['maxillary_score']=m_l_p

df_Osteomeatal.reset_index(inplace=True)

ost_l_p=trained_models['osteomeatal'].predict(df_Osteomeatal)


##concha_prediction

In [120]:
df_concha = pd.DataFrame()

for (i, n) in zip(concha_df_col[1:], concha_data_col[1:]):
  df_concha[i]= X_test[n]

df_concha.reset_index(inplace=True)


concha_p=trained_models['concha'].predict(df_concha)

##nasopharynx_prediction

In [121]:
df_nasopharynx = pd.DataFrame()

for (i, n) in zip(nasopharynx_df_col[1:], nasopharynx_data_col[1:]):
  df_nasopharynx[i]= X_test[n]

df_nasopharynx.reset_index(inplace=True)

naso_p=trained_models['nasopharynx'].predict(df_nasopharynx)

##inferior_prediction

In [122]:
df_inferior = pd.DataFrame()

for (i, n) in zip(inferior_df_col[1:], inferior_data_col[1:]):
  df_inferior[i]= X_test[n]

df_inferior.reset_index(inplace=True)

infer_p=trained_models['inferior'].predict(df_inferior)

##septum_prediction

In [123]:
df_septum = pd.DataFrame()

for (i, n) in zip(septum_df_col[1:], septum_data_col[1:]):
  df_septum[i]= X_test[n]

df_septum['inferior_score']= infer_p
df_septum['concha_score']= concha_p


df_septum.reset_index(inplace=True)


sept_p=trained_models['septum'].predict(df_septum)

##final_prediction_dataset

In [124]:
prediction = pd.DataFrame()
sinuses_score_p=m_r_p+m_l_p+ant_r_p+ant_l_p+post_r_p+post_l_p+sphen_r_p+sphen_l_p+ost_r_p+ost_l_p+f_p
prediction['sinuses_score_p'] = sinuses_score_p
prediction['sinuses_score_p'] = prediction['sinuses_score_p'].apply(lambda x: 0 if x < 18 else 2)
prediction['sergury_p'] = 0
prediction['sept_p']=sept_p
prediction['infer_p']=infer_p
prediction['concha_p']=concha_p
prediction['naso_p']=naso_p
prediction.loc[prediction['sinuses_score_p'] ==18, 'sergury_p'] = 2
prediction.loc[prediction['naso_p'] ==2, 'sergury_p'] = 2
prediction.loc[prediction['infer_p'] ==2, 'sergury_p'] = 2
prediction.loc[prediction['concha_p'] ==2, 'sergury_p'] = 2
prediction.loc[prediction['naso_p'] ==2, 'sergury_p'] = 2

##testing reports

In [ ]:
report = classification_report(X_test['sinuses_score'], prediction['sinuses_score_p'])
print(report)

In [ ]:
report = classification_report(X_test['septum score'], prediction['sept_p'])
print(report)

In [ ]:
report = classification_report(X_test['inferior turbinate score'], prediction['infer_p'])
print(report)

In [ ]:
report = classification_report(X_test['concha bullosa score'], prediction['concha_p'])
print(report)

In [ ]:
report = classification_report(X_test['nasopharynx score'], prediction['naso_p'])
print(report)

##testing AUC

In [125]:
y_p=prediction['sergury_p']
testing_auc = roc_auc_score(Y_test, y_p)
print("testing auc:", testing_auc)

testing auc: 0.6008771929824561


#AUC SCORE

In [126]:
print("trainig auc:", training_auc)
print("testing auc:", testing_auc)

trainig auc: 0.7961348995831755
testing auc: 0.6008771929824561


In [ ]:
with open('models.pkl', 'wb') as f:
    pickle.dump(trained_models, f)